# 数据处理使得FRCN结构能够正确获取数据
## 数据集 ACNE04

In [2]:
import os
from os import listdir
from os.path import join

def create_data_path_file(data_dir, output_file, classes):
    """创建数据路径文件，格式为：图片路径 xmin,ymin,xmax,ymax,class_id ..."""
    with open(output_file, 'w') as f:
        # 直接扫描目录中的所有文件
        for filename in listdir(data_dir):
            if filename.endswith('.jpg'):
                img_path = join(data_dir, filename)
                xml_name = filename.replace('.jpg', '.xml')
                xml_path = join(data_dir, xml_name)
                
                # 检查对应的XML文件是否存在
                if os.path.exists(xml_path):
                    # 解析XML文件，提取类别和边界框信息
                    with open(xml_path, 'r') as xml_file:
                        xml_content = xml_file.read()
                        objects = []
                        # 简单提取类别和边界框信息
                        for obj in xml_content.split('<object>')[1:]:
                            if '</object>' in obj:
                                obj = obj.split('</object>')[0]
                                name_start = obj.find('<name>') + 6
                                name_end = obj.find('</name>')
                                name = obj[name_start:name_end].strip()
                                
                                if name in classes:
                                    xmin_start = obj.find('<xmin>') + 6
                                    xmin_end = obj.find('</xmin>')
                                    xmin = obj[xmin_start:xmin_end].strip()
                                    
                                    ymin_start = obj.find('<ymin>') + 6
                                    ymin_end = obj.find('</ymin>')
                                    ymin = obj[ymin_start:ymin_end].strip()
                                    
                                    xmax_start = obj.find('<xmax>') + 6
                                    xmax_end = obj.find('</xmax>')
                                    xmax = obj[xmax_start:xmax_end].strip()
                                    
                                    ymax_start = obj.find('<ymax>') + 6
                                    ymax_end = obj.find('</ymax>')
                                    ymax = obj[ymax_start:ymax_end].strip()
                                    
                                    objects.append(f"{xmin},{ymin},{xmax},{ymax},{classes.index(name)}")
                        
                        # 写入文件
                        if objects:
                            line = f"{img_path} {' '.join(objects)}\n"
                            f.write(line)

# 在这里设置你的路径
# ---------------------------
train_data_dir = '/data_lg/keru/project/Faster-RCNN-Pytorch/dataset/train'      # 训练集目录路径
val_data_dir = '/data_lg/keru/project/Faster-RCNN-Pytorch/dataset/valid'    # 验证集目录路径
train_output_file = '/data_lg/keru/project/Faster-RCNN-Pytorch/faster-rcnn-pytorch-master/map_out/2007_train.txt'       # 训练集输出文件
val_output_file = '/data_lg/keru/project/Faster-RCNN-Pytorch/faster-rcnn-pytorch-master/map_out/2007_val.txt'           # 验证集输出文件
# ---------------------------

# 指定你的类别
classes = ['nodules and cysts', 'papules', 'pustules', 'whitehead and blackhead']

# 创建训练集文件
create_data_path_file(train_data_dir, train_output_file, classes)

# 创建验证集文件
create_data_path_file(val_data_dir, val_output_file, classes)

print("数据路径文件已创建完成！")

数据路径文件已创建完成！


In [2]:
from PIL import Image

image_path = '/data_lg/keru/project/Faster-RCNN-Pytorch/dataset/train/levle3_144_jpg.rf.cffd1f8ea08ae239330de260360271e2.jpg' # 替换为你的图片路径
image = Image.open(image_path)
width, height = image.size
channels = 3 if image.mode == 'RGB' else 1
print(f'图片的尺寸：{height} x {width}, 通道数：{channels}')

图片的尺寸：640 x 640, 通道数：3


# 验证数据格式是否正确

In [1]:
import os
from PIL import Image

def validate_data_line(line):
    parts = line.strip().split()
    if len(parts) < 2:
        return False, "No bounding boxes found"
    
    image_path = parts[0]
    if not os.path.exists(image_path):
        return False, f"Image file not found: {image_path}"
    
    try:
        img = Image.open(image_path)
        width, height = img.size
    except Exception as e:
        return False, f"Error opening image: {e}"
    
    for bbox_str in parts[1:]:
        bbox_parts = bbox_str.split(',')
        if len(bbox_parts) != 5:
            return False, f"Invalid bbox format: {bbox_str}"
        
        try:
            x1, y1, x2, y2, class_id = map(int, bbox_parts)
        except ValueError:
            return False, f"Non-integer values in bbox: {bbox_str}"
        
        if x1 >= x2 or y1 >= y2:
            return False, f"Invalid coordinates: {bbox_str}"
        
        if x1 < 0 or y1 < 0 or x2 > width or y2 > height:
            return False, f"Coordinates out of image bounds: {bbox_str}"
    
    return True, "Valid"

# 测试单行数据
test_line = "/data_lg/keru/project/Faster-RCNN-Pytorch/dataset/train/levle1_37_jpg.rf.b39582a0736a2bba49e06b2ceae6f986.jpg 215,175,251,210,2 232,444,265,481,2 262,426,296,459,2 287,457,322,487,2 266,483,297,514,2 304,512,336,546,2 320,433,344,453,2 360,444,386,471,2 378,461,409,491,2 405,466,428,495,2 439,428,478,457,2 439,406,465,426,2 418,393,430,412,2 412,436,439,464,2"
valid, message = validate_data_line(test_line)
print(f"Validation result: {valid}, {message}")

Validation result: True, Valid
